In [28]:
import laspy
import numpy as np
import pandas as pd
import os

In [29]:
file_path = r"C:\Users\edwar\Desktop\csv3\Scan Slot 1 No Hysperctral Lights.csv"
df = pd.read_csv(file_path, header=None)  

z = df.to_numpy().flatten().tolist()

x_axis_size = 0.25

y_axis_size = 0.225
x_values = np.arange(0, 10000)  
y_values = np.linspace(-1600, 1600, 3200)  

x = np.repeat(x_values, 3200) * x_axis_size
y = np.tile(y_values, 10000) * y_axis_size


file_path = os.path.join(r"C:\Users\edwar\Desktop\csv3", "point_cloud.bpc")

xyz = np.column_stack((x, y, z)).astype(np.float32)

xyz.tofile(file_path)

print(f"BPC file written: {file_path}")

BPC file written: C:\Users\edwar\Desktop\csv3\point_cloud.bpc


In [30]:
stop

file_path = os.path.join(r"\\192.168.1.100\CoreScan3-2\Acquisitions\RnD\EA", "point_cloud.las")

header = laspy.LasHeader(point_format=1, version="1.2") 
las = laspy.LasData(header)

las.x = np.array(x)
las.y = np.array(y)
las.z = np.array(z)

las.write(file_path)

NameError: name 'stop' is not defined